# Gmail AI-Assistant

### Installing necessary libraries

In [1]:
pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client transformers openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install requests


Defaulting to user installation because normal site-packages is not writeable


### Setting Device to use CPU

In [3]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


Device set to use cpu


### Importing API_Key 

import requests
import os

GROQ_API_KEY = os.getenv("GROQ_API_KEY") or "Put your GROQ_API_KEY here"


### Auttentication for Gmail to use 

In [5]:
import os.path
import base64
import re
from email.mime.text import MIMEText
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Scope to read, send, and modify emails
SCOPES = [
    'https://www.googleapis.com/auth/gmail.readonly',
    'https://www.googleapis.com/auth/gmail.modify',
    'https://www.googleapis.com/auth/gmail.send'
]

#SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


def gmail_authenticate():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('gmail', 'v1', credentials=creds)


### Get and Reading Unread Mails

In [6]:
def get_unread_messages(service):
    results = service.users().messages().list(userId='me', labelIds=['INBOX'], q="is:unread").execute()
    messages = results.get('messages', [])
    return messages

import re

import base64
import re

def get_email_content(service, msg_id):
    msg = service.users().messages().get(userId='me', id=msg_id, format='full').execute()
    headers = msg['payload']['headers']

    subject = next((h['value'] for h in headers if h['name'] == 'Subject'), "(No Subject)")
    sender_raw = next((h['value'] for h in headers if h['name'] == 'From'), "(Unknown Sender)")

    # Extract only the email address
    match = re.search(r'<(.+?)>', sender_raw)
    sender = match.group(1) if match else sender_raw

    # Try getting the body safely
    body = ""
    payload = msg['payload']

    if 'parts' in payload:
        for part in payload['parts']:
            mime_type = part.get('mimeType', '')
            if mime_type == 'text/plain' and 'data' in part.get('body', {}):
                body = base64.urlsafe_b64decode(part['body']['data']).decode('utf-8')
                break
    elif 'data' in payload.get('body', {}):
        body = base64.urlsafe_b64decode(payload['body']['data']).decode('utf-8')
    else:
        body = "(No readable content)"

    return sender, subject, body




def send_reply(service, msg_id, thread_id, to, subject, message_text):
    from email.mime.text import MIMEText
    import base64
    import email

    message = MIMEText(message_text)
    message['to'] = to
    message['subject'] = "Re: " + subject

    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    body = {
        'raw': raw,
        'threadId': thread_id  # ✅ use threadId here
    }

    try:
        return service.users().messages().send(userId='me', body=body).execute()
    except Exception as e:
        print("Error sending email:", e)


### Generating Reply

In [ ]:
def generate_reply(body):
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
    "model": "llama3-8b-8192",
    "messages": [
        {
            "role": "system",
            "content": (
                "You are a helpful, polite assistant who writes short, natural-sounding email replies "
                "as if written by a real person. Do not include phrases like 'Here is a reply' or placeholders like Anees Ur Rehman."
            )
        },
        {
            "role": "user",
            "content": f"Reply to the following email in a natural, professional tone:\n\n{body}"
        }
    ],
    "temperature": 0.7,
    "max_tokens": 150
}


    
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content'].strip()
    except Exception as e:
        print("Groq API error:", e)
        return "Thank you for your message. We'll get back to you shortly."  # Fallback


In [ ]:
def main():
    service = gmail_authenticate()
    messages = get_unread_messages(service)
    print(f"Found {len(messages)} unread emails.")

    for msg in messages:
        msg_id = msg['id']
        message = service.users().messages().get(userId='me', id=msg_id, format='full').execute()
        thread_id = message['threadId']

        sender, subject, body = get_email_content(service, msg_id)

        print(f"Email from {sender}: {subject}")
        print("Body:", body[:100], "...\n")  # Preview body

        # ✅ Skip no-reply emails
        if "no-reply" in sender.lower() or "noreply" in sender.lower():
            print(f"Skipped auto-reply to no-reply sender: {sender}\n")
            continue

        summary = generate_reply(body)


        send_reply(service, msg_id, thread_id, sender, subject, summary)
        print(f"Replied to: {sender}\n")

        service.users().messages().modify(
            userId='me',
            id=msg_id,
            body={'removeLabelIds': ['UNREAD']}
        ).execute()


In [13]:
if __name__ == '__main__':
    main()


Found 4 unread emails.
Email from mranees5323@gmail.com: Re: message for knowing your condition
Body: Glad to hear it.

On Tue, Jun 3, 2025 at 1:40 AM <aneesrehman1747@gmail.com> wrote:

> Hi there, ...

Replied to: mranees5323@gmail.com

Email from no-reply@accounts.google.com: Your Google Account was recovered successfully
Body: [image: Google]
Account recovered successfully


aneesrehman1747@gmail.com
Welcome back to your ...

Skipped auto-reply to no-reply sender: no-reply@accounts.google.com

Email from no-reply@accounts.google.com: Security alert
Body: [image: Google]
Your password was changed


aneesrehman1747@gmail.com
The password for your Goo ...

Skipped auto-reply to no-reply sender: no-reply@accounts.google.com

Email from no-reply@accounts.google.com: Your Google Account was recovered successfully
Body: [image: Google]
Account recovered successfully


aneesrehman1747@gmail.com
Welcome back to your ...

Skipped auto-reply to no-reply sender: no-reply@accounts.google.com

